<a href="https://colab.research.google.com/github/AdrianSosaUV/CartoonPic/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CartoonPic

This Notebook follow and use the code make by Xinrui Wang.

## Clone the Repo

The first step is to clone the repository with the following code:

In [ ]:
!git clone https://github.com/AdrianSosaUV/CartoonPic.git

## Prepare the environment

For this program we will use tensorflow in its version 1.x as follows:

In [ ]:
%tensorflow_version 1.x

You can check it is correct with the following cell:

In [ ]:
import tensorflow as tf
print(tf.__version__)

the next Step its upload your pictures to cartonize to the $test\_ images$ folder

# Load the Program

The next step its load the python script to the environment, use the following code:

In [ ]:
%cd /content/CartoonPic/
import os 
from cartoonize import *
from IPython.display import Image

## Cartonize

Using the following cell, you will cartonize all the photos in the test_image folder (you can see some examples inside), and the results will be in the cartoonized_images folder. Do not forget to download your photos because all the files in the environment will be deleted when your collab session ends.

In [ ]:
model_path = '/content/CartoonPic/saved_models'
load_folder = '/content/CartoonPic/test_images'
save_folder = '/content/CartoonPic/cartoonized_images'
if not os.path.exists(save_folder):
  os.mkdir(save_folder)
cartoonize(load_folder, save_folder, model_path)